In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 7
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'':
  os.chdir(r'')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


C:\Users\kerry\AppData\Local\Temp\ipykernel_42088\3347640643.py:20: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats(fig_format)


{"C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap_external.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\codecs.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\aliases.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\__init__.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\utf_8.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\cp1252.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\abc.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\io.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\li

In [2]:
#| echo: false
import numpy as np
from cvxopt import matrix
from cvxopt.solvers import qp as Solver, options as SolverOptions
from scipy.optimize import minimize_scalar

SolverOptions["show_progress"] = False

class portfolio:
    def __init__(self, means, cov, Shorts):
        self.means = np.array(means)
        self.cov = np.array(cov)
        self.Shorts = Shorts
        self.n = len(means)
        if Shorts:
            w = np.linalg.solve(cov, np.ones(self.n))
            self.GMV = w / np.sum(w)
            w = np.linalg.solve(cov, means)
            self.piMu = w / np.sum(w)
        else:
            n = self.n
            Q = matrix(cov, tc="d")
            p = matrix(np.zeros(n), (n, 1), tc="d")
            G = matrix(-np.identity(n), tc="d")
            h = matrix(np.zeros(n), (n, 1), tc="d")
            A = matrix(np.ones(n), (1, n), tc="d")
            b = matrix([1], (1, 1), tc="d")
            sol = Solver(Q, p, G, h, A, b)
            self.GMV = np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])

    def frontier(self, m):
        if self.Shorts:
            gmv = self.GMV
            piMu = self.piMu
            m1 = gmv @ self.means
            m2 = piMu @ self.means
            a = (m - m2) / (m1 - m2)
            return a * gmv + (1 - a) * piMu
        else:
            n = self.n
            Q = matrix(self.cov, tc="d")
            p = matrix(np.zeros(n), (n, 1), tc="d")
            G = matrix(-np.identity(n), tc="d")
            h = matrix(np.zeros(n), (n, 1), tc="d")
            A = matrix(np.vstack((np.ones(n), self.means)), (2, n), tc="d")
            b = matrix([1, m], (2, 1), tc="d")
            sol = Solver(Q, p, G, h, A, b)
            return np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])

    def tangency(self, r):
        if self.Shorts:
            w = np.linalg.solve(self.cov, self.means - r)
            return w / np.sum(w)
        else:
            def f(m):
                w = self.frontier(m)
                mn = w @ self.means
                sd = np.sqrt(w.T @ self.cov @ w)
                return - (mn - r) / sd
            m = minimize_scalar(f, bounds=[max(r, np.min(self.means)), max(r, np.max(self.means))], method="bounded").x
            return self.frontier(m)

    def optimal(self, raver, rs=None, rb=None):
        n = self.n
        if self.Shorts:
            if (rs or rs==0) and (rb or rb==0):
                Q = np.zeros((n + 2, n + 2))
                Q[2:, 2:] = raver * self.cov
                Q = matrix(Q, tc="d")
                p = np.array([-rs, rb] + list(-self.means))
                p = matrix(p, (n + 2, 1), tc="d")
                G = np.zeros((2, n + 2))
                G[0, 0] = G[1, 1] = -1
                G = matrix(G, (2, n+2), tc="d")
                h = matrix([0, 0], (2, 1), tc="d")
                A = matrix([1, -1] + n*[1], (1, n+2), tc="d")
                b = matrix([1], (1, 1), tc="d")
                sol = Solver(Q, p, G, h, A, b)
                return np.array(sol["x"]).flatten()[2:] if sol["status"] == "optimal" else None
            else:
                w = np.linalg.solve(self.cov, self.means)
                a = np.sum(w)
                return (a/raver)*self.piMu + (1-a/raver)*self.GMV
        else:
           if (rs or rs==0) and (rb or rb==0):
                Q = np.zeros((n + 2, n + 2))
                Q[2:, 2:] = raver * self.cov
                Q = matrix(Q, tc="d")
                p = np.array([-rs, rb] + list(-self.means))
                p = matrix(p, (n+2, 1), tc="d")
                G = matrix(-np.identity(n + 2), tc="d")
                h = matrix(np.zeros(n+2), (n+2, 1), tc="d")
                A = matrix([1, -1] + n * [1], (1, n+2), tc="d")
                b = matrix([1], (1, 1), tc="d")
                sol = Solver(Q, p, G, h, A, b)
                return np.array(sol["x"]).flatten()[2:] if sol["status"] == "optimal" else None
           else:
                Q = matrix(raver * self.cov, tc="d")
                p = matrix(-self.means, (n, 1), tc="d")
                G = matrix(-np.identity(n), tc="d")
                h = matrix(np.zeros(n), (n, 1), tc="d")
                A = matrix(np.ones(n), (1, n), tc="d")
                b = matrix([1], (1, 1), tc="d")
                sol = Solver(Q, p, G, h, A, b)
                return np.array(sol["x"]).flatten() if sol["status"] == "optimal" else None

In [3]:
#| label: fig-two-asset
#| fig-cap: Portfolios of two risky assets
import numpy as np
import pandas as pd
import plotly.graph_objects as go

#Based on French data library return series
# Asset 1: US Equity
# Asset 2: Developed International
# Asset 1: Emerging Market
mn1= 6
mn2= 6.5
sd1= 15
sd2= 16.5 
corr12 = 75 


def data(mn1, mn2, sd1, sd2, c):
    c = c / 100
    mns = [mn1, mn2]
    sds = [sd1, sd2]
    grid = np.linspace(0, 1, 101)
    ports = [np.array([w, 1 - w]) for w in grid]
    means = [p.T @ np.array(mns) for p in ports]
    df = pd.DataFrame(means)
    df.columns = ["mean"]
    cov = np.array(
        [[sds[0] ** 2, sds[0] * sds[1] * c], [sds[0] * sds[1] * c, sds[1] ** 2]]
    ).reshape(2, 2)
    df["stdev"] = [np.sqrt(p.T @ cov @ p) for p in ports]
    df["wt1"] = grid
    df["wt2"] = 1 - df.wt1
    for col in ["mean", "stdev"]:
        df[col] = df[col] / 100
    return df


def figtbl(mn1, mn2, sd1, sd2, c,asset1_name, asset2_name):

    df = data(mn1, mn2, sd1, sd2, c)
    string0 = "Correlation = " +str(np.round(c,1)) + "%"
    trace0 = go.Scatter(
        x=df["stdev"],
        y=df["mean"],
        mode="lines",
        text=100 * df["wt1"],
        customdata=100 * df["wt2"],
        hovertemplate=string0 + "<br>"+asset1_name + ": %{text:.0f}%<br>"+asset2_name + ": %{customdata:.0f}%<extra></extra>",
        name="Frontier: " + asset1_name + " + " + asset2_name, 
        line=dict(color="red"), 
    )

    # Plot the two assets
    df = df[df.wt1.isin([0, 1])]
    df["text"] = np.where(df.wt1 == 1, asset1_name, asset2_name)
    trace1 = go.Scatter(
        x=df[df.wt1==1]["stdev"],
        y=df[df.wt1==1]["mean"],
        mode="markers",
        text=df[df.wt1==1]["text"],
        hovertemplate="%{text}<extra></extra>",
        marker=dict(size=15, color="black"),
        name=asset1_name
    )
    trace2 = go.Scatter(
        x=df[df.wt1==0]["stdev"],
        y=df[df.wt1==0]["mean"],
        mode="markers",
        text=df[df.wt1==0]["text"],
        hovertemplate="%{text}<extra></extra>",
        marker=dict(size=15, color="blue"),
        name=asset2_name
    )


    fig = go.Figure()
    fig.add_trace(trace0)
    fig.add_trace(trace1)
    fig.add_trace(trace2)
    fig.layout.xaxis["title"] = "Standard Deviation"
    fig.layout.yaxis["title"] = "Expected Return"
    fig.update_xaxes(range=[0.85 * df["stdev"].min(), 1.15 * df["stdev"].max()])
    fig.update_yaxes(range=[0.85 * df["mean"].min(), 1.15 * df["mean"].max()])
    fig.update_yaxes(tickformat=".1%")
    fig.update_xaxes(tickformat=".1%")
    fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
    fig.show()
figtbl(mn1,mn2,sd1,sd2,corr12,"US Equity", "Developed Intl")

In [4]:
#| label: fig-three-asset
#| fig-cap: Portfolios of three risky assets
import pandas as pd
import numpy as np
from scipy.stats import uniform
import plotly.graph_objects as go

#Based on French return series
# Asset 1: US Equity
# Asset 2: Developed International
# Asset 1: Emerging Market
mn1= 6
mn2= 6.5
mn3= 8
sd1= 15
sd2= 16.5 
sd3= 21
corr12 = 75 
corr13 = 75 
corr23 = 75 

def random_wts(num):
    w = uniform.rvs(0, 1, num)
    return w / w.sum()

ports = [random_wts(3) for i in range(3000)]
ports1 = [(0, x, 1 - x) for x in np.linspace(0, 1, 21)]
ports2 = [(x, 0, 1 - x) for x in np.linspace(0, 1, 21)]
ports3 = [(x, 1 - x, 0) for x in np.linspace(0, 1, 21)]
ports = ports + ports1 + ports2 + ports3

ports23 = ports1
ports13 = ports2
ports12 = ports3

def is_pos_def(x):
    if np.all(np.linalg.eigvals(x) > 0):
        return 'True'
    else:
        return 'False'

def data(mn1, mn2, mn3, sd1, sd2, sd3, c12, c13, c23):
    mns = pd.Series(np.array([mn1, mn2, mn3]), index=["stock1", "stock2", "stock3"])
    sds = np.array([sd1, sd2, sd3])
    C = np.diag([1.0, 1.0, 1.0])
    C[0, 1] = c12 / 100
    C[0, 2] = c13 / 100
    C[1, 0] = c12 / 100
    C[1, 2] = c23 / 100
    C[2, 0] = c13 / 100
    C[2, 1] = c23 / 100
    D = np.diag(sds / 100)
    C = D @ C @ D
    
    # Random investment opportunity set (3-asset)
    df = pd.DataFrame(
        dtype=float,
        index=range(len(ports)),
        columns=["mean", "stdev", "wt1", "wt2", "wt3"],
    )
    df["mean"] = [p @ mns / 100 for p in ports]
    df["stdev"] = [np.sqrt(p @ C @ p) for p in ports]
    df["wt1"] = [100 * p[0] for p in ports]
    df["wt2"] = [100 * p[1] for p in ports]
    df["wt3"] = [100 * p[2] for p in ports]
    df = df.sort_values(by="mean")
    
    # Dataframes of 2-asset portfolios
    df12 = pd.DataFrame(dtype=float,index=range(len(ports12)),\
                    columns=['mean','stdev','wt1','wt2','wt3'])
    df12['mean'] = [p @ mns/100 for p in ports12]
    df12['stdev'] = [np.sqrt(p @ C @ p) for p in ports12]
    df12['wt1'] = [100*p[0] for p in ports12]
    df12['wt2'] = [100*p[1] for p in ports12]
    df12['wt3'] = [100*p[2] for p in ports12]
    df12 = df12.sort_values(by='mean')

    df13 = pd.DataFrame(dtype=float,index=range(len(ports13)),\
                    columns=['mean','stdev','wt1','wt2','wt3'])
    df13['mean'] = [p @ mns/100 for p in ports13]
    df13['stdev'] = [np.sqrt(p @ C @ p) for p in ports13]
    df13['wt1'] = [100*p[0] for p in ports13]
    df13['wt2'] = [100*p[1] for p in ports13]
    df13['wt3'] = [100*p[2] for p in ports13]
    df13 = df13.sort_values(by='mean')

    df23 = pd.DataFrame(dtype=float,index=range(len(ports23)),\
                    columns=['mean','stdev','wt1','wt2','wt3'])
    df23['mean'] = [p @ mns/100 for p in ports23]
    df23['stdev'] = [np.sqrt(p @ C @ p) for p in ports23]
    df23['wt1'] = [100*p[0] for p in ports23]
    df23['wt2'] = [100*p[1] for p in ports23]
    df23['wt3'] = [100*p[2] for p in ports23]
    df23 = df23.sort_values(by='mean')   
    return df, mns / 100, sds / 100, C, df12, df13, df23


def figtbl(mn1, mn2, mn3, sd1, sd2, sd3, c12, c13, c23, asset1_name, asset2_name, asset3_name):
    df, mns, sds, C, df12, df13, df23 = data(mn1, mn2, mn3, sd1, sd2, sd3, c12, c13, c23)
    cd = np.empty(shape=(df.shape[0], 3, 1), dtype=float)
    cd[:, 0] = np.array(df.wt1).reshape(-1, 1)
    cd[:, 1] = np.array(df.wt2).reshape(-1, 1)
    cd[:, 2] = np.array(df.wt3).reshape(-1, 1)
    string =  asset1_name +": %{customdata[0]:.0f}%<br>"
    string += asset2_name +": %{customdata[1]:.0f}%<br>"
    string += asset3_name +": %{customdata[2]:.0f}%<br>"
    string += "<extra></extra>"
    trace1 = go.Scatter(
        x=df["stdev"], y=df["mean"], mode="markers", customdata=cd, hovertemplate=string, name="Random Portfolios"
    )
    trace2 = go.Scatter(
        x=sds,
        y=mns,
        mode="markers",
        text=[asset1_name,asset2_name,asset3_name],
        hovertemplate="%{text}<extra></extra>",
        marker=dict(size=15, color="red"),
        name="Underlying Assets"
    )
    
    #--------------
    # 2-asset frontiers
    #--------------
    #12
    cd12 = np.empty(shape=(df12.shape[0],3,1), dtype=float)
    cd12[:,0] = np.array(df12.wt1).reshape(-1,1)
    cd12[:,1] = np.array(df12.wt2).reshape(-1,1)
    cd12[:,2] = np.array(df12.wt3).reshape(-1,1)
    string12 = asset1_name +": %{customdata[0]:.0f}%<br>"
    string12+= asset2_name +": %{customdata[1]:.0f}%<br>"
    string12+= asset3_name +": %{customdata[2]:.0f}%<br>"
    string12+= '<extra></extra>'
    trace12 = go.Scatter(x=df12['stdev'],y=df12['mean'],mode='lines',customdata=cd12,hovertemplate=string12, line=dict(color="red"), 
        name = "Frontier: " + asset1_name + " + " + asset2_name)

    #23
    cd23 = np.empty(shape=(df23.shape[0],3,1), dtype=float)
    cd23[:,0] = np.array(df23.wt1).reshape(-1,1)
    cd23[:,1] = np.array(df23.wt2).reshape(-1,1)
    cd23[:,2] = np.array(df23.wt3).reshape(-1,1)
    string23 = asset1_name +": %{customdata[0]:.0f}%<br>"
    string23+= asset2_name +": %{customdata[1]:.0f}%<br>"
    string23+= asset3_name +": %{customdata[2]:.0f}%<br>"
    string23+= '<extra></extra>'
    trace23 = go.Scatter(x=df23['stdev'],y=df23['mean'],mode='lines',customdata=cd23,hovertemplate=string23, line=dict(color="black"), 
        name = "Frontier: " + asset2_name + " + "  + asset3_name)

    #13
    cd13 = np.empty(shape=(df13.shape[0],3,1), dtype=float)
    cd13[:,0] = np.array(df13.wt1).reshape(-1,1)
    cd13[:,1] = np.array(df13.wt2).reshape(-1,1)
    cd13[:,2] = np.array(df13.wt3).reshape(-1,1)
    string13 = asset1_name +": %{customdata[0]:.0f}%<br>"
    string13+= asset2_name +": %{customdata[1]:.0f}%<br>"
    string13+= asset3_name +": %{customdata[2]:.0f}%<br>"
    string13+= '<extra></extra>'
    trace13 = go.Scatter(x=df13['stdev'],y=df13['mean'],mode='lines',customdata=cd13,hovertemplate=string13, line=dict(color="orange"), 
        name = "Frontier: " + asset1_name + " + " + asset3_name)

    fig = go.Figure()
    fig.add_trace(trace1)
    fig.add_trace(trace2)
    fig.add_trace(trace12)
    fig.add_trace(trace23)
    fig.add_trace(trace13)
    fig.layout.xaxis["title"] = "Standard Deviation"
    fig.layout.yaxis["title"] = "Expected Return"
    fig.update_xaxes(range=[0.85 * df["stdev"].min(), 1.15 * df["stdev"].max()])
    fig.update_yaxes(range=[0.85 * df["mean"].min(),  1.15 * df["mean"].max()])    
    fig.update_yaxes(tickformat=".1%")
    fig.update_xaxes(tickformat=".1%")
    fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
    fig.show()
    # return largefig(fig), is_pos_def(C)
figtbl(mn1, mn2, mn3, sd1, sd2, sd3, corr12, corr13, corr23, "US Equity", "Developed Intl", "Emerging Mkt")

In [5]:
#| label: fig-diversification
#| fig-cap: Portfolios of three risky assets
import pandas as pd
import numpy as np
import plotly.graph_objects as go

std  = 40
corr0 = 10
corr1 = 25
corr2 = 50

# Range of x-axis
Num = 200

def data(std, cor):
    var = std ** 2
    cov = var * cor
    stdevs = [np.sqrt(var / n + (n - 1) * cov / n) for n in range(1, Num + 1)]
    df = pd.DataFrame(stdevs)
    df.columns = ["Standard Deviation"]
    df["Number of Assets"] = [i for i in range(1, Num + 1)]
    return df
def figtbl(std, cor0, cor1, cor2):
    # The inputs are in percents
    std /= 100
    cor0 /= 100
    cor1 /= 100
    cor2 /= 100

    # Correlation 0
    df = data(std, cor0)
    trace0 = go.Scatter(
        x=df["Number of Assets"],
        y=df["Standard Deviation"],
        mode="lines",
        hovertemplate="Correlation=" + f'{100*cor0:.0f}%' + "<br>Number of Assets = %{x}<br>Standard Deviation = %{y:0.2%} <extra></extra>",
        name = "Correlation=" + f'{100*cor0:.0f}%'
    )

    # Correlation 1
    df = data(std, cor1)
    trace1 = go.Scatter(
        x=df["Number of Assets"],
        y=df["Standard Deviation"],
        mode="lines",
        hovertemplate="Correlation=" + f'{100*cor1:.0f}%' + "<br>Number of Assets = %{x}<br>Standard Deviation = %{y:0.2%} <extra></extra>",
        name = "Correlation=" + f'{100*cor1:.0f}%'
    )   
    # Correlation 2
    df = data(std, cor2)
    trace2 = go.Scatter(
        x=df["Number of Assets"],
        y=df["Standard Deviation"],
        mode="lines",
        hovertemplate="Correlation=" + f'{100*cor2:.0f}%' + "<br>Number of Assets = %{x}<br>Standard Deviation = %{y:0.2%} <extra></extra>",
        name = "Correlation=" + f'{100*cor2:.0f}%'
    )      

    fig = go.Figure()
    fig.add_trace(trace0)
    fig.add_trace(trace1)
    fig.add_trace(trace2)
    fig.update_yaxes(tickformat=",.0%", rangemode="tozero")
    fig.layout.xaxis["title"] = "Number of Assets"
    fig.layout.yaxis["title"] = "Standard Deviation"
    fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.74))
    fig.show()
    
figtbl(std, corr0, corr1, corr2)

In [6]:
#| label: fig-two-asset-shorting
#| fig-cap: Portfolios of two risky assets with shorting
import numpy as np
import pandas as pd
import plotly.graph_objects as go

#Based on French data library return series
# Asset 1: US Equity
# Asset 2: Developed International
# Asset 1: Emerging Market
mn1= 6
mn2= 6.5
sd1= 15
sd2= 16.5 
corr12 = 75 


def data(mn1, mn2, sd1, sd2, c):
    c = c / 100
    mns = [mn1, mn2]
    sds = [sd1, sd2]
    grid = np.linspace(-0.5, 1.5, 201)
    ports = [np.array([w, 1 - w]) for w in grid]
    means = [p.T @ np.array(mns) for p in ports]
    df = pd.DataFrame(means)
    df.columns = ["mean"]
    cov = np.array(
        [[sds[0] ** 2, sds[0] * sds[1] * c], [sds[0] * sds[1] * c, sds[1] ** 2]]
    ).reshape(2, 2)
    df["stdev"] = [np.sqrt(p.T @ cov @ p) for p in ports]
    df["wt1"] = grid
    df["wt2"] = 1 - df.wt1
    for col in ["mean", "stdev"]:
        df[col] = df[col] / 100
    return df


def figtbl(mn1, mn2, sd1, sd2, c,asset1_name, asset2_name):

    df = data(mn1, mn2, sd1, sd2, c)
    string0 = "Correlation = " +str(np.round(c,1)) + "%"
    trace0 = go.Scatter(
        x=df["stdev"],
        y=df["mean"],
        mode="lines",
        text=100 * df["wt1"],
        customdata=100 * df["wt2"],
        hovertemplate=string0 + "<br>"+asset1_name + ": %{text:.0f}%<br>"+asset2_name + ": %{customdata:.0f}%<extra></extra>",
        name="Frontier: " + asset1_name + " + " + asset2_name, 
        line=dict(color="red"), 
    )

    # Plot the two assets
    df = df[df.wt1.isin([0, 1])]
    df["text"] = np.where(df.wt1 == 1, asset1_name, asset2_name)
    trace1 = go.Scatter(
        x=df[df.wt1==1]["stdev"],
        y=df[df.wt1==1]["mean"],
        mode="markers",
        text=df[df.wt1==1]["text"],
        hovertemplate="%{text}<extra></extra>",
        marker=dict(size=15, color="black"),
        name=asset1_name
    )
    trace2 = go.Scatter(
        x=df[df.wt1==0]["stdev"],
        y=df[df.wt1==0]["mean"],
        mode="markers",
        text=df[df.wt1==0]["text"],
        hovertemplate="%{text}<extra></extra>",
        marker=dict(size=15, color="blue"),
        name=asset2_name
    )


    fig = go.Figure()
    fig.add_trace(trace0)
    fig.add_trace(trace1)
    fig.add_trace(trace2)
    fig.layout.xaxis["title"] = "Standard Deviation"
    fig.layout.yaxis["title"] = "Expected Return"
    fig.update_xaxes(range=[0.85 * df["stdev"].min(), 1.15 * df["stdev"].max()])
    fig.update_yaxes(range=[0.85 * df["mean"].min(), 1.15 * df["mean"].max()])
    fig.update_yaxes(tickformat=".1%")
    fig.update_xaxes(tickformat=".1%")
    fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
    fig.show()
figtbl(mn1,mn2,sd1,sd2,corr12,"US Equity", "Developed Intl")

In [7]:
#| label: fig-frontier-gmv
#| fig-cap: The Efficient Frontier and the Global Minimum Variance Portfolio
import numpy as np
import plotly.graph_objects as go

#Based on French return series
# Asset 1: US Equity
# Asset 2: Developed International
# Asset 1: Emerging Market
mn1= 6
mn2= 6.5
mn3= 8
sd1= 15
sd2= 16.5 
sd3= 21
corr12 = 75 
corr13 = 75 
corr23 = 75 

def is_pos_def(x):
    if np.all(np.linalg.eigvals(x) > 0):
        return 'True'
    else:
        return 'False'
    
def figtbl(mn1, mn2, mn3, sd1, sd2, sd3, c12, c13, c23, asset_names):
    mns = np.array((mn1, mn2, mn3)) / 100
    sds = np.array((sd1, sd2, sd3)) / 100
    corr = np.identity(3)
    corr[0,1] = corr[1,0] = c12 / 100
    corr[0,2] = corr[2,0] = c13 / 100
    corr[1,2] = corr[2,1] = c23 / 100
    cov = np.diag(sds) @ corr @ np.diag(sds)
    N = 3

    def custom(string, ports):
        cd = np.empty(shape=(len(ports), N, 1), dtype=float)
        for i in range(N):
            cd[:, i] = np.array([w[i] for w in ports]).reshape(-1, 1)
        string += "<br>"
        for i in range(N):
            string += asset_names[i]
            string += ": %{customdata["
            string += str(i)
            string += "]:.1%}<br>"
        string += "<extra></extra>"
        return string, cd

    P = portfolio(mns, cov, True)

    mingrid = 0.9*np.min(mns)
    maxgrid = 1.025*np.max(mns)
    mnsFrontier = np.linspace(mingrid, maxgrid, 50)
    portsFrontier = [P.frontier(m) for m in mnsFrontier]
    sdsFrontier = [np.sqrt(w @ cov @ w) for w in portsFrontier]


    string, cd = custom('frontier', portsFrontier)
    # string, cd = custom('frontier w/o short sales', portsFrontier)
    trace1 = go.Scatter(
        x=sdsFrontier,
        y=mnsFrontier,
        mode="lines",
        customdata=cd,
        hovertemplate=string,
        line=dict(color="blue"),
        name="3-Fund Frontier"
    )

    gmv = P.GMV

    string = "GMV portfolio<br>"
    for i in range(N):
        string += asset_names[i]+": " + f'{gmv[i]:.1%}' + "<br>"
    string += "<extra></extra>"
    trace1a = go.Scatter(
        x=[np.sqrt(gmv @ cov @ gmv)],
        y=[gmv @ mns],
        mode="markers",
        hovertemplate=string,        
        marker=dict(size=15, color="blue"),
        name="Global Minimum Variance Portfolio"
    )

    trace3 = go.Scatter(
        x=sds,
        y=mns,
        # text=[1, 2, 3],
        # hovertemplate="Asset %{text}<extra></extra>",
        text=asset_names,
        hovertemplate="%{text}<extra></extra>",
        mode="markers",
        marker=dict(size=15, color="red"),
        name="Underlying Assets"
    )
 
    fig = go.Figure()  
    for trace in (trace1, trace1a, trace3):
        fig.add_trace(trace)
    #trace1=frontier w/o short sales
    #trace1a=GMV portfolio w/o short sales
    #trace3=Assets
        
        
    fig.layout.xaxis["title"] = "Standard Deviation"
    fig.layout.yaxis["title"] = "Expected Return"
    # fig.update_xaxes(range=[0, 1.25 * np.max(sds)])
    # fig.update_yaxes(range=[0, 1.25 * np.max(mns)])
    fig.update_yaxes(tickformat=".1%")
    fig.update_xaxes(tickformat=".1%")
    fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
    fig.show()
    # return largefig(fig), is_pos_def(cov)
figtbl(mn1, mn2, mn3, sd1, sd2, sd3, corr12, corr13, corr23, ["US Equity", "Developed Intl", "Emerging Mkt"])